### project3_4_KNN.ipynb, 

wherein I try using K Nearest Neighbors to see if I can do better. It didn't turn out to be successful.

It wouldn't have been as useful for my project anyway, since it doesn't have the same interpretability as logistic regression.

In [3]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier



Load in my data, and clean it as before (cells copied):

In [4]:
df = pd.read_csv('./data/redditcomments.csv')
df.drop(columns='Unnamed: 0',inplace=True)
df.dropna(inplace=True)
df['char_length'] = [len(comment) for comment in list(df['comment'])]
df = df[(df['char_length']>=10) & (df['char_length']<1000)]
df.head()

,comment,date,types,char_length
0,I think it's the end of the first episode rath...,1615999558,fantasy,76
1,I think you mean the prologue of the Wheel of ...,1615999475,fantasy,87
2,"Not properly ""Fantasy"", but ""Horus Rising"" ope...",1615999395,fantasy,218
3,Love that trilogy. Bit slow to get started (af...,1615999383,fantasy,85
4,&gt;At this point I think Sanderson being the ...,1615999332,fantasy,566


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18746 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   comment      18746 non-null  object
 1   date         18746 non-null  int64 
 2   types        18746 non-null  object
 3   char_length  18746 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 732.3+ KB


In [7]:
comments = list(df['comment'])
df['flagged'] = [1 if ('removed' in comment.split(' ')) else 0 for comment in comments]
df = df[df['flagged']==0]

Load in the other dataframes. Note that the scores dataframe now includes the Ridge and LASSO regulated models.

In [8]:
stops_df = pd.read_csv('./data/stopwords.csv')
stops = list(stops_df['0'])
stops

['thick',
 'beyond',
 'six',
 'hereafter',
 'both',
 'her',
 'am',
 'forty',
 'very',
 'its',
 'above',
 'ours',
 'who',
 'someone',
 'might',
 'whenever',
 'per',
 'un',
 'may',
 'cannot',
 'formerly',
 'before',
 'up',
 'of',
 'amongst',
 'couldnt',
 'more',
 'four',
 'it',
 'thereupon',
 'there',
 'again',
 'him',
 'beside',
 'they',
 'less',
 'us',
 'hasnt',
 'in',
 'while',
 'one',
 'seeming',
 'had',
 'several',
 'thereafter',
 'since',
 'ten',
 'for',
 'too',
 'been',
 'de',
 'etc',
 'herself',
 'wherein',
 'herein',
 'during',
 'such',
 'noone',
 'yours',
 'serious',
 'whether',
 'enough',
 'whoever',
 'cant',
 'must',
 'eg',
 'otherwise',
 'itself',
 'get',
 'them',
 'why',
 'few',
 'nobody',
 'rather',
 'between',
 'ourselves',
 'fill',
 'below',
 'are',
 'indeed',
 'mine',
 'still',
 'perhaps',
 'thence',
 'most',
 'others',
 'no',
 'go',
 'then',
 'inc',
 'have',
 'hers',
 'twenty',
 'this',
 'full',
 'me',
 'either',
 'side',
 'onto',
 'five',
 'latterly',
 'con',
 'except

In [9]:
predictions = pd.read_csv('./data/predictions.csv')
predictions.drop(columns = 'Unnamed: 0', inplace=True)
predictions.head()

,types,Simple NB,Simple LR,LR 5000 features
0,fantasy,fantasy,fantasy,fantasy
1,fantasy,fantasy,fantasy,fantasy
2,scifi,scifi,scifi,scifi
3,scifi,scifi,scifi,scifi
4,scifi,scifi,scifi,scifi


In [10]:
predicted_probabilities = pd.read_csv('./data/predicted_probabilities.csv')
predicted_probabilities.drop(columns = 'Unnamed: 0', inplace=True)
predicted_probabilities.head()

,types,prediction,err type,err label,probability
0,0,0,0,"fantasy, correct",2.081928e-01
1,0,0,0,"fantasy, correct",2.242084e-08
2,1,1,3,"scifi, correct",9.309696e-01
3,1,1,3,"scifi, correct",9.962833e-01
4,1,1,3,"scifi, correct",7.649797e-01


In [16]:
scores = pd.read_csv('./data/scores.csv')


In [18]:
scores

,model,train,test
0,Baseline,0.501809,0.501909
1,NBayesian,0.829448,0.779184
2,LogReg,0.940715,0.814949
3,LogReg 5000,0.894628,0.801085
4,"LR with Lasso, a=1",0.892867,0.789237
5,"LR with Lasso, a=10",0.977844,0.796527
6,"LR with Ridge a=1 (ran 0.1, 1, 10)",0.951472,0.806389


Make X & y, test train split, and count vectorize with my stop words list:

In [19]:
X = df['comment']
y = df['types']
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42, stratify=y)
cv = CountVectorizer(stop_words=stops)
cv.fit(X_train,y_train)
Xvec_train = cv.transform(X_train)
Xvec_test  = cv.transform(X_test)



In [21]:
knnscores = []

k_list = [1, 3, 5, 25, 501]
weight_list = ['uniform', 'distance']

for weight in weight_list:
    for k in k_list:
        knn = KNeighborsClassifier(n_neighbors = k, weights = weight)
        knn.fit(Xvec_train,y_train)
        score_train = knn.score(Xvec_train, y_train)
        score_test  = knn.score(Xvec_test,  y_test)
        if k<10:    model_name = f'knn, k={k},   {weight}'
        elif k<100: model_name = f'knn, k={k},  {weight}'
        else:       model_name = f'knn, k={k}, {weight}'
        print(model_name)
        print(f'Train: {score_train}, Test: {score_test}')
        knnscores.append({'model': model_name, 'train': score_train,   'test' : score_test})

knn, k=1,   uniform
Train: 0.9931389365351629, Test: 0.6102058319039451
knn, k=3,   uniform
Train: 0.7667953116066324, Test: 0.6078473413379074
knn, k=5,   uniform
Train: 0.7612206975414523, Test: 0.6189965694682675
knn, k=25,  uniform
Train: 0.6666666666666666, Test: 0.5919811320754716
knn, k=501, uniform
Train: 0.508218982275586, Test: 0.5098627787307033
knn, k=1,   distance
Train: 0.9931389365351629, Test: 0.6102058319039451
knn, k=3,   distance
Train: 0.9933533447684391, Test: 0.6106346483704974
knn, k=5,   distance
Train: 0.9934248141795312, Test: 0.6204974271012007
knn, k=25,  distance
Train: 0.9934248141795312, Test: 0.5951972555746141
knn, k=501, distance
Train: 0.9934248141795312, Test: 0.5201543739279588


With uniform weighting, the model is extremely overfit for small k and approaches the baseline accuracy for large k.

With distance weighting, it's overfit for all tested values of k.

There doesn't seem to be any evidence of a "sweet spot" that would reduce bias and variance and beat my simple logistic regression model with the test score.

Save these scores in my dataframe:

In [22]:
knnscores_df = pd.DataFrame(knnscores)
knnscores_df['ndx'] = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
knnscores_df

,model,train,test,ndx
0,"knn, k=1, uniform",0.993139,0.610206,7
1,"knn, k=3, uniform",0.766795,0.607847,8
2,"knn, k=5, uniform",0.761221,0.618997,9
3,"knn, k=25, uniform",0.666667,0.591981,10
4,"knn, k=501, uniform",0.508219,0.509863,11
5,"knn, k=1, distance",0.993139,0.610206,12
6,"knn, k=3, distance",0.993353,0.610635,13
7,"knn, k=5, distance",0.993425,0.620497,14
8,"knn, k=25, distance",0.993425,0.595197,15
9,"knn, k=501, distance",0.993425,0.520154,16


In [65]:
scores['ndx'] = range(7)
scores

,model,train,test,ndx
0,Baseline,0.501809,0.501909,0
1,NBayesian,0.829448,0.779184,1
2,LogReg,0.940715,0.814949,2
3,LogReg 5000,0.894628,0.801085,3
4,"LR with Lasso, a=1",0.892867,0.789237,4
5,"LR with Lasso, a=10",0.977844,0.796527,5
6,"LR with Ridge a=1 (ran 0.1, 1, 10)",0.951472,0.806389,6


In [66]:
scores_df= scores.append(knnscores_df)
scores_df.set_index('ndx',inplace=True)
scores_df

,model,train,test
ndx,,,
0,Baseline,0.501809,0.501909
1,NBayesian,0.829448,0.779184
2,LogReg,0.940715,0.814949
3,LogReg 5000,0.894628,0.801085
4,"LR with Lasso, a=1",0.892867,0.789237
5,"LR with Lasso, a=10",0.977844,0.796527
6,"LR with Ridge a=1 (ran 0.1, 1, 10)",0.951472,0.806389
7,"knn, k=1, uniform",0.993139,0.610206
8,"knn, k=3, uniform",0.766795,0.607847


In [67]:

scores_df.to_csv('./data/scores.csv')

I have now tried Naive Bayesian with the default parameters, Logistic Regression with the default parameters, Logistic Regression variations in a pipeline with changes to the max number of features and to the n-gram ranges, Logistic Regression with Ridge and LASSO regularization, and KNN with both uniform and distance weighting and with the number of neighbors ranging from 1 to 501.

In terms of accuracy on the test data, the Logistic Regression model with the default parameters continues to be the best. It is overfit, but still has an accuracy of 80% on the test data.

All of these are using count vectorization. Next up I will try Tfid vectorization, but I will stick with the logistic regression.

### Further notebooks:

* project3_5: Tfid instead of CountVectorizer, and experiments with stemming/lemmatizing.

* project3_6: Random Forest models.

* project3_7: Enough modeling, time to visualize.


